#### Import

In [1]:
import numpy as np
import numba as nb
from scipy.stats import gaussian_kde

#### Functions

In [31]:
@nb.njit(nb.float64(nb.float64))
def get_phi(x):
    return np.exp(-x**2 / 2) / np.sqrt(2*np.pi)


@nb.njit(nb.float64(nb.float64[:]))
def silverman_bandwidth(data):
    if len(data) < 2:
        raise ValueError("Need at least 2 data points")
    
    std = np.std(data)
    IQR = np.quantile(data, .75) - np.quantile(data, .25)
    A = min(std, IQR / 1.34)
    return 0.9 * A * len(data)**(-0.2) / std


@nb.njit(nb.float64(nb.float64[:], nb.float64))
def gauss_kde(data, point):
    n = len(data)
    h = silverman_bandwidth(data)
    result = 0
    for i in range(n):
        result += get_phi((point - data[i]) / h) / (n * h)
    return result

#### Test

In [32]:
n = 10
np.random.seed(1000002)
data = np.random.randn(n)
point = .7

In [33]:
bw = silverman_bandwidth(data)
print(bw)

gaussian_kde(data, bw_method=bw)(point)

0.4565397115427201


array([0.46061493])

In [34]:
gauss_kde(data, point)

0.44433341511978275

In [10]:
def gaussian_kernel_density_estimate(x, data, h):
    n = len(data)
    sum_term = np.sum(np.exp(-0.5 * ((x - data) / h) ** 2) / (np.sqrt(2 * np.pi) * h))
    return sum_term / n

In [18]:
gaussian_kernel_density_estimate(point, data, bw)

11.037829153014503

In [17]:
def gaussian_kernel_density_estimate(x, data, h):
    # Calculate Gaussian kernel
    kernel = np.exp(-0.5 * ((x - data) / h)) / (np.sqrt(2 * np.pi) * h)
    # Normalize by dividing by the sum of the kernel values
    normalization_factor = np.sum(kernel)
    return np.sum(kernel)